In [1]:
import xml.etree.ElementTree as ET
import simplekml
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon

def extract_coordinates_from_kml(file_path):
    """
    Extracts coordinates from a KML (Keyhole Markup Language) file.

    Args:
        file_path (str): The path to the KML file.

    Returns:
        list: A list of tuples representing latitude and longitude coordinates.
    """

    # Create an ElementTree from the KML file
    tree = ET.parse(file_path)
    root = tree.getroot()

    # KML Namespace
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    # Find Placemark elements that contain coordinates
    placemarks = root.findall('.//kml:Placemark', ns)

    # List to store coordinates
    coordinates = []

    # Iterate over placemarks and extract coordinates
    for placemark in placemarks:
        # Find the element containing coordinates
        coordinates_element = placemark.find('.//kml:coordinates', ns)

        # Extract coordinates from the text of the element
        if coordinates_element is not None and coordinates_element.text:
            coordinates_str = coordinates_element.text.strip()
            # Convert coordinates string to a list of tuples
            coordinates.extend([tuple(map(float, coord.split(','))) for coord in coordinates_str.split()])

    return coordinates


def filter_coordinates_inside_boundaries(coordinate_farm, sample_dataframe):
    """
    Filters coordinates within the specified farm boundaries.

    Args:
        coordinate_farm (list of tuples): List of tuples representing the farm coordinates (latitude, longitude).
        sample_dataframe (pd.DataFrame): DataFrame containing 'latitude' and 'longitude' columns.

    Returns:
        pd.DataFrame: Filtered DataFrame containing coordinates within the farm boundaries.
    """

    # Create a polygon based on farm coordinates
    farm_polygon = Polygon(coordinate_farm)

    # Convert DataFrame coordinates into Point objects
    sample_points = [Point(longitude, latitude) for latitude, longitude in zip(sample_dataframe['latitude'], sample_dataframe['longitude'])]

    # Check if each point is inside the farm polygon
    within_farm = [farm_polygon.contains(point) for point in sample_points]

    # Filter the original DataFrame based on the condition
    filtered_dataframe = sample_dataframe[within_farm].reset_index(drop=True)

    # Print information about the filtering
    print(f'Total points before filtering: {len(sample_dataframe)}')
    print(f'Total points inside the farm: {len(filtered_dataframe)}')
    
    return filtered_dataframe


def create_kml_from_dataframe(dataframe, kml_file_path, style_id='inline80', placemark_name='Placemark'):
    """
    Creates a KML file from a DataFrame containing latitude and longitude coordinates.

    Args:
        dataframe (pd.DataFrame): DataFrame containing 'latitude' and 'longitude' columns.
        kml_file_path (str): The path to save the generated KML file.
        style_id (str): Style ID for KML Placemark (default is 'inline80').
        placemark_name (str): Name for each Placemark (default is 'Placemark').

    Returns:
        None
    """

    # Create a KML object
    kml = simplekml.Kml()

    # Create a style with the given id
    style = simplekml.Style()
    style.linestyle.color = simplekml.Color.blue
    style.linestyle.width = 2
    style.polystyle.fill = 0

    # Add coordinates from the DataFrame as points with the specified style
    for _, row in dataframe.iterrows():
        point = kml.newpoint(name=placemark_name, coords=[(row['longitude'], row['latitude'])])
        point.style = style

    # Save the KML file
    kml.save(kml_file_path)
    print(f'KML file saved at: {kml_file_path}')



In [3]:
kml_file_name = 'RV-T85.kml'
map1_kml_file_path = 'data/map1/kml'

map1_df = pd.read_csv('data/map1/raw/map1.csv')

map1_boundaries_coordinates = extract_coordinates_from_kml(f'{map1_kml_file_path}/{kml_file_name}')
map1_boundaries_coordinates = np.array(map1_boundaries_coordinates)[:, :2]

map1_filtered_df = filter_coordinates_inside_boundaries(map1_boundaries_coordinates, map1_df)

# save df to csv
map1_filtered_df.to_csv('data/map1/raw/map1_filtered.csv', index=False)

create_kml_from_dataframe(map1_filtered_df, 'data/map1/kml/map1.kml')

map1_filtered_df

Total points before filtering: 59752
Total points inside the farm: 9455
KML file saved at: data/map1/kml/map1.kml


,latitude,longitude,B01.tiff,B02.tiff,B03.tiff,B04.tiff,B05.tiff,B06.tiff,B07.tiff,B08.tiff,B09.tiff,B10.tiff,B11.tiff,B12.tiff,B8A.tiff
0,-23.191156,-54.438359,0.1275,0.1005,0.1050,0.1019,0.1537,0.2568,0.3090,0.2879,0.0406,0.0013,0.2998,0.1957,0.3267
1,-23.191156,-54.438269,0.1278,0.1030,0.1055,0.1170,0.1562,0.2499,0.3000,0.2709,0.0406,0.0013,0.3028,0.1976,0.3247
2,-23.191156,-54.438179,0.1280,0.1039,0.1067,0.1148,0.1573,0.2497,0.3006,0.2877,0.0407,0.0013,0.3053,0.1996,0.3269
3,-23.191156,-54.438089,0.1282,0.1057,0.1067,0.1156,0.1567,0.2588,0.3148,0.3018,0.0407,0.0013,0.3078,0.2022,0.3346
4,-23.191156,-54.437999,0.1282,0.1093,0.1077,0.1270,0.1564,0.2646,0.3231,0.2906,0.0408,0.0013,0.3091,0.2044,0.3390
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9450,-23.200720,-54.431618,0.1309,0.1180,0.1295,0.1708,0.1805,0.2721,0.3290,0.3022,0.0428,0.0015,0.3444,0.2462,0.3532
9451,-23.200720,-54.431528,0.1309,0.1165,0.1255,0.1577,0.1779,0.2720,0.3284,0.3028,0.0423,0.0015,0.3395,0.2420,0.3520
9452,-23.200720,-54.431439,0.1309,0.1139,0.1193,0.1491,0.1781,0.2701,0.3236,0.3082,0.0417,0.0015,0.3363,0.2384,0.3453
9453,-23.200720,-54.431349,0.1308,0.1065,0.1149,0.1350,0.1838,0.2672,0.3179,0.2874,0.0412,0.0015,0.3355,0.2441,0.3384


In [ ]:

# Name and path of the KML file
kml_file_name = 'Pivot.kml'
kml_file_path = 'data/map2/kml'

map2_df = pd.read_csv('data/map2/raw/map2.csv')


# Extract coordinates from the KML file
map2_boundaries_coordinates = extract_coordinates_from_kml(f'{kml_file_path}/{kml_file_name}')

# Convert the coordinates to a numpy array and keep only latitude and longitude columns
map2_boundaries_coordinates = np.array(map2_boundaries_coordinates)[:, :2]

# Filter the coordinates of map2_df based on the provided boundaries_coordinates
map2_filtered_df = filter_coordinates_inside_boundaries(map2_boundaries_coordinates, map2_df)

# Save the filtered DataFrame to a CSV file
map2_filtered_df.to_csv('data/map2/raw/map2_filtered.csv', index=False)

# Creating a KML file from the filtered DataFrame and saving it to the specified path
create_kml_from_dataframe(map2_filtered_df, 'data/map2/kml/map2.kml')

map2_filtered_df

Total points before filtering: 59752
Total points inside the farm: 9349
KML file saved at: data/map2/kml/map2.kml


,latitude,longitude,B01.tiff,B02.tiff,B03.tiff,B04.tiff,B05.tiff,B06.tiff,B07.tiff,B08.tiff,B8A.tiff,B09.tiff,B11.tiff,B12.tiff
0,-22.318380,-52.669486,0.0231,0.0217,0.0485,0.0192,0.0810,0.4249,0.5612,0.5866,0.5825,0.5640,0.2404,0.1137
1,-22.318380,-52.669396,0.0234,0.0230,0.0487,0.0206,0.0812,0.4225,0.5547,0.5836,0.5804,0.5619,0.2403,0.1144
2,-22.318380,-52.669307,0.0237,0.0220,0.0483,0.0192,0.0815,0.4208,0.5498,0.5887,0.5791,0.5571,0.2407,0.1158
3,-22.318380,-52.669217,0.0240,0.0212,0.0472,0.0201,0.0817,0.4198,0.5483,0.5828,0.5758,0.5506,0.2406,0.1175
4,-22.318380,-52.669127,0.0244,0.0214,0.0493,0.0220,0.0820,0.4185,0.5481,0.5756,0.5702,0.5430,0.2404,0.1198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9344,-22.327342,-52.668498,0.0325,0.0265,0.0529,0.0356,0.1088,0.4134,0.5280,0.5704,0.5561,0.4947,0.2968,0.1779
9345,-22.327342,-52.668408,0.0324,0.0262,0.0534,0.0362,0.1106,0.4131,0.5322,0.5660,0.5577,0.4930,0.2942,0.1772
9346,-22.327342,-52.668318,0.0323,0.0256,0.0538,0.0357,0.1103,0.4078,0.5286,0.5518,0.5527,0.4924,0.2938,0.1785
9347,-22.327342,-52.668228,0.0321,0.0283,0.0564,0.0442,0.1109,0.3992,0.5176,0.5359,0.5413,0.4927,0.2950,0.1817
